In [6]:
from sqlitedict import SqliteDict

path_dict = 'tmp.sqlite'
with SqliteDict(path_dict) as tmp_dict:
    tmp = {}
    tmp['a'] = [1,2,'a']
    tmp['b'] = 1
    tmp_dict['a'] = [tmp]
    tmp_dict.commit()
with SqliteDict(path_dict) as tmp_dict:
    print(tmp_dict['a'])

[{'a': [1, 2, 'a'], 'b': 1}]


In [5]:
import random
from random import shuffle
import spacy

from utils_db import mkdir_if_not_exist, dict_load_json, dict_save_json

class ClaimDatabaseStage2:
    def __init__(self, path_database_dir, path_raw_data, method_false_statements, fraction_validation):
        # description: create a database in which the 
        
        # folder layout:
        # base_folder_name
        # - settings.json
        # - train
            # - evidence_correct
            # - evidence_incorrect
            # - combined
        # - validation
            # - evidence_correct
            # - evidence_incorrect
            # - combined 
        # - dev
            # - evidence_correct
            # - evidence_incorrect
            # - combined
        
        self.path_database_dir = path_database_dir
        self.path_raw_data = path_raw_data
        self.method_false_statements = method_false_statements
        self.fraction_validation = fraction_validation
        
        self.total_nr_claims_train_set = self.get_nr_claims(data_set_type = 'train')
        self.total_nr_claims_dev_set = self.get_nr_claims(data_set_type = 'dev')
#         self.nr_claims_training = 
#         self.nr_claims_validation = 
#         self.nr_claims_dev = 
        self.random_seed_nr = 1

        base_folder_name = 'claim_database'
        self.path_base_folder_dir = os.path.join(self.path_database_dir, base_folder_name)
        self.path_settings = os.path.join(self.path_base_folder_dir, 'settings.json')
        
        self.nlp = spacy.load('en', disable=["parser", "ner"])
        self.partition = self.get_partition_list(fraction_validation = fraction_validation)
        
#         if not os.path.isdir(self.path_base_folder_dir):
#             mkdir_if_not_exist(self.path_base_folder_dir)
#             self.settings = {}
            
            
#             data_set_type_list = ['training', 'validation', 'dev']
            
#             for data_set_type in data_set_type_list:
#                 data = self.get_data_dict(data_set_type)
#                 self.save_data_dict(self, data)
#                 data = None
                
#             dict_save_json(self.settings, self.path_settings)
#         else:
#             self.settings = dict_load_json(self.path_settings)

    def get_nr_claims(self, data_set_type):
        path_data_set = os.path.join(self.path_raw_data, data_set_type + '.jsonl')
        claim_dict_list = load_jsonl(path_data_set)
        nr_claims = len(claim_dict_list)
        return nr_claims
    
    def get_partition_list(self, fraction_validation):
        list_total_shuffled = list(range(self.total_nr_claims_train_set))
        random.seed(self.random_seed_nr)
        shuffle(list_total_shuffled)
        
        partition = {}
        partition['train'] = list_total_shuffled[int(self.total_nr_claims_train_set*fraction_validation):self.total_nr_claims_train_set]
        partition['validation'] = list_total_shuffled[0:int(self.total_nr_claims_train_set*fraction_validation)]
        partition['dev'] = list(range(self.total_nr_claims_dev_set))

        return partition
        
#     def create_database(self):
        
#     def get_data_dict(self, data_set_type):
#         # description
#         # input:
#         # - data_set_type: 'train', 'validation' or 'dev' [str]
        
#         path_data_type_dir = os.path.join(self.path_base_folder_dir, data_set_type)
        
#         return data
        
#     def save_data_dict(self, data):


In [22]:
from wiki_database import WikiDatabaseSqlite
from utils_db import mkdir_if_not_exist
import config
import os

path_wiki_pages = os.path.join(config.ROOT, config.DATA_DIR, config.WIKI_PAGES_DIR, 'wiki-pages')
path_wiki_database_dir = os.path.join(config.ROOT, 'tmp_wiki_database')
# mkdir_if_not_exist(path_wiki_database_dir)
wiki_database = WikiDatabaseSqlite(path_wiki_database_dir, path_wiki_pages)


wiki_database
- Load existing settings file
- Load title dictionary


In [7]:
import os

from utils_db import load_jsonl

import config

path_dir_database = 'claim_db'
path_raw_data = os.path.join(config.ROOT, config.DATA_DIR, config.RAW_DATA_DIR)
claim_data_set = 'train'
fraction_validation = 0.1

claim_database = ClaimDatabaseStage2(path_dir_database, path_raw_data, claim_data_set, fraction_validation)


In [8]:
data_set_type = 'train'
path_data_set = os.path.join(claim_database.path_raw_data, data_set_type + '.jsonl')
claim_dict_list = load_jsonl(path_data_set)


In [71]:
claim_dict_list

[{'id': 75397,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
  'evidence': [[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
    [92206, 104971, 'Fox_Broadcasting_Company', 0]]]},
 {'id': 150448,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'Roman Atwood is a content creator.',
  'evidence': [[[174271, 187498, 'Roman_Atwood', 1]],
   [[174271, 187499, 'Roman_Atwood', 3]]]},
 {'id': 214861,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.',
  'evidence': [[[255136, 254645, 'History_of_art', 2]]]},
 {'id': 156709,
  'verifiable': 'VERIFIABLE',
  'label': 'REFUTES',
  'claim': 'Adrienne Bailon is an accountant.',
  'evidence': [[[180804, 193183, 'Adrienne_Bailon', 0]]]},
 {'id': 83235,
  'verifiable': 'NOT VERIFIABLE',
  'label'

In [93]:
import spacy

from wiki_database import Text

from utils_wiki_database import normalise_text

method_tokenization = 'tokenize_text_pos'

claim_nr = 9

dict_out = {}
claim_dict = claim_dict_list[claim_nr]
# doc = claim_database.nlp(claim_dict['claim'])

claim_dict

sentence_dict = {}
for interpreter in claim_dict['evidence']:
    for proof in interpreter:
        title = proof[2]
        if title is not None:
            normalised_title = normalise_text(title)
            line_nr = proof[3]
            evidence_sentence = wiki_database.get_lines_from_title(normalised_title, [line_nr])[0]
            if normalised_title in sentence_dict:
                sentence_dict[normalised_title].append(line_nr)
            else:
                sentence_dict[normalised_title] = [line_nr]
sentence_dict
for title, sentences_correct_list in sentence_dict.items():
    
#         get_cosine(text1, text2)

{'The Ten Commandments (1956 film)': [0, 20]}

In [ ]:
# save observation, n correct, n closest
def get_claim_dict(claim_dict, wiki_database)
    sentence_dict = {}
    for interpreter in claim_dict['evidence']:
        for proof in interpreter:
            title = proof[2]
            if title is not None:
                normalised_title = normalise_text(title)
                line_nr = proof[3]
                evidence_sentence = wiki_database.get_lines_from_title(normalised_title, [line_nr])[0]
                if normalised_title in sentence_dict:
                    sentence_dict[normalised_title].append(line_nr)
                else:
                    sentence_dict[normalised_title] = [line_nr]
                    
    correct_evidence_list = []
    incorrect_evidence_list = []
    
    for title, sentences_correct_list in sentence_dict.items():
        
        for sentence_nr in sentences_correct_list:
            # correct
            dict_out = {}
            text_claim = claim_dict['claim']
            tag_list, word_list = get_word_tag_list_from_text(text_str = text_claim, nlp = claim_database.nlp, method_tokenization_str = method_tokenization)
            dict_out['premises'] = word_list
            dict_out['ids'] = claim_dict['id']
            dict_out['labels'] = claim_dict['label']
            dict_out['primises_tags'] = tag_list
            
            dict_out['hypotheses'] = wiki_database.get_line_from_title(title, line_nr)
            
            # incorrect
            lines_file = wiki_database.get_line_from_title(title, line_nr)
            
    return correct_evidence_list, incorrect_evidence_list



In [31]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(text1, text2):
    vec1 = text_to_vector(text1)
    vec2 = text_to_vector(text2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

def get_word_tag_list_from_text(text_str, nlp, method_tokenization_str):
    doc = nlp(text_str)
    text = Text(doc)
    delimiter = text.delimiter_position_tag
    tokenized_text_list = text.process([method_tokenization_str])
    word_list = []
    tag_list = []

    for i in range(len(tokenized_text_list)):
        key = tokenized_text_list[i]
        tag, word = key.split(delimiter)
        if i == 0:
            if not(tag == 'PROPN'):
                word = word.lower()
        word_list.append(word)
        tag_list.append(tag)
    
    return tag_list, word_list


In [105]:
claim_dict_list[0]

{'id': 75397,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'evidence': [[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
   [92206, 104971, 'Fox_Broadcasting_Company', 0]]]}

In [ ]:
# get sentence (title, id)

wiki_database
- Load existing settings file
- Load title dictionary


In [1]:
from utils_wiki_database import normalise_text
title = '1992_Northwestern_Wildcats_football_team'
wiki_database.get_lines_from_title(normalise_text(title), [1,2 ])


NameError: name 'wiki_database' is not defined

In [5]:
# Folder Structure
# database_stage_2
# label_is_evidence
# label_is_not_evidence
# combined equal
# combined not equal

# for every claim:
# every evidence sentence -> label is_evidence
# other sentences in document -> label is_not_evidence
# Two folders

SyntaxError: invalid syntax (<ipython-input-5-4ad11a0dfd66>, line 4)

In [ ]:
{"ids": ids,
"premises": premises,
"hypotheses": hypotheses,
"labels": labels}